# Lab 5


Matrix Representation: In this lab you will be creating a simple linear algebra system. In memory, we will represent matrices as nested python lists as we have done in lecture. In the exercises below, you are required to explicitly test every feature you implement, demonstrating it works.

1. Create a `matrix` class with the following properties:
    * It can be initialized in 2 ways:
        1. with arguments `n` and `m`, the size of the matrix. A newly instanciated matrix will contain all zeros.
        2. with a list of lists of values. Note that since we are using lists of lists to implement matrices, it is possible that not all rows have the same number of columns. Test explicitly that the matrix is properly specified.
    * Matrix instances `M` can be indexed with `M[i][j]` and `M[i,j]`.
    * Matrix assignment works in 2 ways:
        1. If `M_1` and `M_2` are `matrix` instances `M_1=M_2` sets the values of `M_1` to those of `M_2`, if they are the same size. Error otherwise.
        2. In example above `M_2` can be a list of lists of correct size.


In [40]:
class Matrix:
    def __init__(self, n, m=None):
        if isinstance(n, list):
            self.data = n
            self.n = len(n)
            self.m = len(n[0]) if n else 0
            if not all(len(row) == self.m for row in n):
                raise ValueError("All rows must have the same number of columns.")
        else:
            self.n = n
            self.m = m
            self.data = [[0] * m for _ in range(n)]

    def __getitem__(self, key):
        if isinstance(key, tuple):
            return self.data[key[0]][key[1]]
        return self.data[key]

    def __setitem__(self, key, value):
        if isinstance(key, tuple):
            self.data[key[0]][key[1]] = value
        else:
            self.data[key] = value

    def shape(self):
        return self.n, self.m

    def transpose(self):
        return Matrix([[self.data[j][i] for j in range(self.n)] for i in range(self.m)])

    def row(self, n):
        return Matrix([self.data[n]])

    def column(self, n):
        return Matrix([[self.data[i][n]] for i in range(self.n)])

    def to_list(self):
        return self.data

    def block(self, n_0, n_1, m_0, m_1):
        return Matrix([row[m_0:m_1] for row in self.data[n_0:n_1]])

    def scalarmul(self, c):
        return Matrix([[c * self.data[i][j] for j in range(self.m)] for i in range(self.n)])

    def add(self, N):
        if self.shape() != N.shape():
            raise ValueError("Matrix dimensions must match for addition.")
        return Matrix([[self.data[i][j] + N.data[i][j] for j in range(self.m)] for i in range(self.n)])

    def sub(self, N):
        if self.shape() != N.shape():
            raise ValueError("Matrix dimensions must match for subtraction.")
        return Matrix([[self.data[i][j] - N.data[i][j] for j in range(self.m)] for i in range(self.n)])

    def mat_mult(self, N):
        if self.m != N.n:
            raise ValueError("Matrix multiplication dimension mismatch.")
        return Matrix([[sum(self.data[i][k] * N.data[k][j] for k in range(self.m)) for j in range(N.m)] for i in range(self.n)])

    def element_mult(self, N):
        if self.shape() != N.shape():
            raise ValueError("Matrix dimensions must match for element-wise multiplication.")
        return Matrix([[self.data[i][j] * N.data[i][j] for j in range(self.m)] for i in range(self.n)])

    def equals(self, N):
        return self.data == N.to_list()

    def __eq__(self, N):
        return self.equals(N)

    def __mul__(self, other):
        if isinstance(other, (int, float)):
            return self.scalarmul(other)
        elif isinstance(other, Matrix):
            return self.mat_mult(other)
        else:
            raise TypeError("Unsupported operand type for *: 'Matrix' and '{}'".format(type(other).__name__))

    def __rmul__(self, other):
        return self * other

    def __add__(self, other):
        return self.add(other)

    def __sub__(self, other):
        return self.sub(other)

    def __repr__(self):
        return "\n".join(str(row) for row in self.data)


2. Add the following methods:
    * `shape()`: returns a tuple `(n,m)` of the shape of the matrix.
    * `transpose()`: returns a new matrix instance which is the transpose of the matrix.
    * `row(n)` and `column(n)`: that return the nth row or column of the matrix M as a new appropriately shaped matrix object.
    * `to_list()`: which returns the matrix as a list of lists.
    *  `block(n_0,n_1,m_0,m_1)` that returns a smaller matrix located at the n_0 to n_1 columns and m_0 to m_1 rows. 
    * (Extra credit) Modify `__getitem__` implemented above to support slicing.
        

In [41]:
# Answer to 2: Matrix Methods
def shape(self):
    return self.n, self.m

def transpose(self):
    return Matrix([[self.data[j][i] for j in range(self.n)] for i in range(self.m)])

def row(self, n):
    return Matrix([self.data[n]])

def column(self, n):
    return Matrix([[self.data[i][n]] for i in range(self.n)])

def to_list(self):
    return self.data

def block(self, n_0, n_1, m_0, m_1):
    return Matrix([row[m_0:m_1] for row in self.data[n_0:n_1]])



3. Write functions that create special matrices (note these are standalone functions, not member functions of your `matrix` class):
    * `constant(n,m,c)`: returns a `n` by `m` matrix filled with floats of value `c`.
    * `zeros(n,m)` and `ones(n,m)`: return `n` by `m` matrices filled with floats of value `0` and `1`, respectively.
    * `eye(n)`: returns the n by n identity matrix.

In [42]:
def constant(n, m, c):
    return Matrix([[c] * m for _ in range(n)])

def zeros(n, m):
    return constant(n, m, 0)

def ones(n, m):
    return constant(n, m, 1)

def eye(n):
    return Matrix([[1 if i == j else 0 for j in range(n)] for i in range(n)])


4. Add the following member functions to your class. Make sure to appropriately test the dimensions of the matrices to make sure the operations are correct.
    * `M.scalarmul(c)`: a matrix that is scalar product $cM$, where every element of $M$ is multiplied by $c$.
    * `M.add(N)`: adds two matrices $M$ and $N$. Don’t forget to test that the sizes of the matrices are compatible for this and all other operations.
    * `M.sub(N)`: subtracts two matrices $M$ and $N$.
    * `M.mat_mult(N)`: returns a matrix that is the matrix product of two matrices $M$ and $N$.
    * `M.element_mult(N)`: returns a matrix that is the element-wise product of two matrices $M$ and $N$.
    * `M.equals(N)`: returns true/false if $M==N$.

In [43]:
def scalarmul(self, c):
    return Matrix([[c * self.data[i][j] for j in range(self.m)] for i in range(self.n)])

def add(self, N):
    if self.shape() != N.shape():
        raise ValueError("Matrix dimensions must match for addition.")
    return Matrix([[self.data[i][j] + N[i, j] for j in range(self.m)] for i in range(self.n)])

def sub(self, N):
    if self.shape() != N.shape():
        raise ValueError("Matrix dimensions must match for subtraction.")
    return Matrix([[self.data[i][j] - N[i, j] for j in range(self.m)] for i in range(self.n)])

def mat_mult(self, N):
    if self.m != N.n:
        raise ValueError("Matrix multiplication dimension mismatch.")
    return Matrix([[sum(self.data[i][k] * N[k, j] for k in range(self.m)) for j in range(N.m)] for i in range(self.n)])

def element_mult(self, N):
    if self.shape() != N.shape():
        raise ValueError("Matrix dimensions must match for element-wise multiplication.")
    return Matrix([[self.data[i][j] * N[i, j] for j in range(self.m)] for i in range(self.n)])

def equals(self, N):
    return self.data == N.to_list()


5. Overload python operators to appropriately use your functions in 4 and allow expressions like:
    * 2*M
    * M*2
    * M+N
    * M-N
    * M*N
    * M==N
    * M=N


In [44]:

def __eq__(self, N):
    return self.equals(N)

def __mul__(self, other):
    if isinstance(other, (int, float)):
        return self.scalarmul(other)
    return self.mat_mult(other)

def __rmul__(self, other):
    return self * other

def __add__(self, other):
    return self.add(other)

def __sub__(self, other):
    return self.sub(other)

def __repr__(self):
    return "\n".join(str(row) for row in self.data)


6. Demonstrate the basic properties of matrices with your matrix class by creating two 2 by 2 example matrices using your Matrix class and illustrating the following:

$$
(AB)C=A(BC)
$$
$$
A(B+C)=AB+AC
$$
$$
AB\neq BA
$$
$$
AI=A
$$

In [46]:
# Answer to 6: Demonstrating basic properties
A = Matrix([[1, 2], [3, 4]])
B = Matrix([[5, 6], [7, 8]])
C = Matrix([[2, 3], [4, 5]])
I = eye(2)

# (AB)C = A(BC)
result_1 =( A * B) * C == A * (B * C)
print("(AB)C = A(BC):", result_1)

# A(B + C) = AB + AC
result_2 = A * (B + C) == (A * B) + (A * C)
print("A(B + C) = AB + AC:", result_2)

# AB ≠ BA
result_3 = A * B != B * A
print("AB ≠ BA:", result_3)

# AI = A
result_4 = A * I == A
print("AI = A:", result_4)


(AB)C = A(BC): True
A(B + C) = AB + AC: True
AB ≠ BA: True
AI = A: True


In [47]:
#Write a function make_deck that returns a list of all of the cards in a standard card deck. 
#The return should be a list of tuples of pairs of suit and value. 
#For example the 10 of Clubs would be ('Clubs', 10) and Queen of Hearts would be ('Hearts', 'Queen'). Recall that a deck has 52 cards, divided into 4 suits (Clubs, Diamonds, Hearts, and Spades), and that each suit has 13 cards: 2 to 10, Jack, Queen, King, and Ace. 




def make_deck():
    suits = ["Clubs", "Diamonds", "Hearts", "Spades"]
    values = [2, 3, 4, 5, 6, 7, 8, 9, 10, "Jack", "Queen", "King", "Ace"]
    return [(suit, value) for suit in suits for value in values]

# Example usage
deck = make_deck()
print(deck)


[('Clubs', 2), ('Clubs', 3), ('Clubs', 4), ('Clubs', 5), ('Clubs', 6), ('Clubs', 7), ('Clubs', 8), ('Clubs', 9), ('Clubs', 10), ('Clubs', 'Jack'), ('Clubs', 'Queen'), ('Clubs', 'King'), ('Clubs', 'Ace'), ('Diamonds', 2), ('Diamonds', 3), ('Diamonds', 4), ('Diamonds', 5), ('Diamonds', 6), ('Diamonds', 7), ('Diamonds', 8), ('Diamonds', 9), ('Diamonds', 10), ('Diamonds', 'Jack'), ('Diamonds', 'Queen'), ('Diamonds', 'King'), ('Diamonds', 'Ace'), ('Hearts', 2), ('Hearts', 3), ('Hearts', 4), ('Hearts', 5), ('Hearts', 6), ('Hearts', 7), ('Hearts', 8), ('Hearts', 9), ('Hearts', 10), ('Hearts', 'Jack'), ('Hearts', 'Queen'), ('Hearts', 'King'), ('Hearts', 'Ace'), ('Spades', 2), ('Spades', 3), ('Spades', 4), ('Spades', 5), ('Spades', 6), ('Spades', 7), ('Spades', 8), ('Spades', 9), ('Spades', 10), ('Spades', 'Jack'), ('Spades', 'Queen'), ('Spades', 'King'), ('Spades', 'Ace')]
